### 主程式
1. 設定資料庫 & 爬蟲設定
2. 顯示UI
3. 使用者操作
4. 讀取資料庫顯示在UI上

In [1]:
from sqlDB import sqlDB
from crawler3 import cwb
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta


from ipywidgets.widgets import Button, DatePicker, Dropdown, Label
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display, clear_output

# 設定下載日期範圍
start_date = date(2021,9,1)
end_date =date.today()
# 設定下載路徑
downloadFolder = r'C:\Downloads\cwb'
# 設定想要下載的測站
targetList = []
targetList.append('福山植物園')

# 創立物件 (cwb類別)
cwbInstance = cwb()
# 創建物件 (sqlDB類別)
dbInstance = sqlDB("cwb")


資料夾存在
Database Opened successfully
County, Table existed
Station, Table does not exist
Station, Table created


In [2]:
def show_Widget():
    ### Box1, Box2
    # 初始化 Button
    btn_initial = Button(
    description ='初始化',
    icon='bug',
    button_style='warning',
    layout=Layout(width='100px')
    )
    # 下載 Button
    btn_download = Button(
        description ='下載',
        icon='download',
        button_style='warning',
        layout=Layout(width='100px')
    )
    # 顯示Table
    btn_table = Button(
        description ='表格',
        icon='table',
        button_style='warning',
        layout=Layout(width='100px')
    )
    # 顯示曲線圖
    btn_chart = Button(
        description ='曲線圖',
        icon='line-chart',
        button_style='warning',
        layout=Layout(width='100px')
    )
    
    # 下拉式選單 Dropdown
    drop_county = Dropdown(
        options=dbInstance.getTableList("County"),
        description='城市',
        disabled = False,
        layout=Layout(width='auto', align_items='center')

    )
    # 下拉式選單 Dropdown
    drop_station = Dropdown(
        options = dbInstance.getStationListatCounty("Station",0),
        description='測站',
        disabled = False,
        layout=Layout(width='auto', align_items='center')
    )
    
    # 選擇日期
    date_picker_from = DatePicker(
        description='日期從',
        disabled=False,
        value = datetime.now().date()-relativedelta(months=1)-timedelta(days=1)
    )
    # 選擇日期
    date_picker_to = DatePicker(
        description='日期至',
        disabled=True,
        value = datetime.now().date()-timedelta(days=1)
    )
    
    # Button click Event
    def initial_click(b):
        cwbInstance.openBrowser()
        cwbInstance.getAllSelect()
        print('創建物件耶耶 上網瞜!!!')
        # 放置資料庫的Table (County)
        dbInstance.deleteTableAllRows("County")
        dbInstance.insertCountyTable(cwbInstance)
        # 放置資料庫的Table (Station)
        dbInstance.deleteTableAllRows("Station")
        dbInstance.insertStationTable(cwbInstance)
        # 清除畫面 重新顯示
        clear_output()
        show_Widget()
        
    def download_click(b):
        #clear_output()
        # 取得選取CountyIndex
        countyIndex = drop_county.options[drop_county.value][1] 
        # 取得選取SationName
        stationName = drop_station.value
        # 去資料庫撈取StationCode
        print(countyIndex)
        print(stationName)
        dbInstance.cursorObj.execute(f" Select StationCode from Station where CountyIndex={countyIndex} and StationName Like '%{stationName}%'")
        print(dbInstance.cursorObj.fetchall())
        
        # 清除畫面 重新顯示
        #clear_output()
        #show_Widget()
        
        
        
    
    # Dropdown Event
    def show_station(a):
        drop_station.options = dbInstance.getStationListatCounty("Station", drop_county.value)    
    
    btn_initial.on_click(initial_click)
    btn_download.on_click(download_click)
    drop_county.observe(show_station)
    
    # https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Flexible_Box_Layout/Aligning_Items_in_a_Flex_Container
    box_layout_column = Layout(width='80%', display='flex', flex_flow='column', justify_content='space-around', align_items='flex-start')
    box_layout_column2 = Layout(width='80%', display='flex', flex_flow='column', justify_content='space-around', align_items='center')
    
    box_layout_row = Layout(width='80%', display='flex', flex_flow='row', justify_content='space-between', align_items='center')
    
    V_items1 = [drop_county, drop_station]
    V_items2 = [date_picker_from, date_picker_to]
    
    V_items3 = [btn_initial, btn_download]
    V_items4 = [btn_table, btn_chart]
    
    H_items1 = [VBox(V_items1, layout = box_layout_column), VBox(V_items3, layout = box_layout_column)]
    H_items2 = [VBox(V_items2, layout = box_layout_column), VBox(V_items4, layout = box_layout_column)]

    Box1 = HBox(H_items1, layout = box_layout_row)
    Box2 = HBox(H_items2, layout = box_layout_row)

    display(VBox([Box1,Box2]),layout = box_layout_column)
    


In [3]:
show_Widget()

In [ ]:
cwbInstance.downloadFolder = downloadFolder
cwbInstance.downloadList(targetList, start_date, end_date)

In [ ]:
cwbInstance.allStations

In [ ]:
# 開啟下載的csv整理後存入db裏頭
import pandas as pd
filepath = "C:\Downloads\cwb\新北市_福山植物園 (Fushan Botanical Garden)_C0AH90\C0AH90-2021-09-05.csv"
df = pd.read_csv(filepath, parse_dates=True)
df